In [101]:
import os.path
import pickle
from dataclasses import dataclass
from datetime import datetime, timedelta

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [102]:
SCOPES = ["https://www.googleapis.com/auth/calendar"]
CREDENTIALS_FILE = "../../../../credentials.json"
PICKLE_FILE = "../../../../token.pickle"

In [103]:
# startTime/endTime format: "yyyy-MM-ddTHH:mm:ss+01:00"
# date format: "yyyy-MM-dd"
@dataclass
class Event:
    title: str
    description: str
    location: str
    full_day: bool
    date: str
    startTime: str
    endTime: str

In [104]:
def get_calendar_service():
    creds = None
    if os.path.exists(PICKLE_FILE):  # stores access tokens once created (automatically)
        with open(PICKLE_FILE, "rb") as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(PICKLE_FILE, "wb") as token:
            pickle.dump(creds, token)

    service = build("calendar", "v3", credentials=creds)
    return service

In [105]:
# timestamp format: "yyyy-MM-ddTHH:mm:ss.ffffffZ"
def getEventsByTimeframe(min_timestamp: str, max_timestamp: str):
    service = get_calendar_service()

    calendars = service.calendarList().list().execute().get('items', [])
    calendarIDs = []
    eventArray = []
    for i in range(len(calendars)):
        currentCalendar = calendars[i].get('id', '')
        if (currentCalendar != ''):

            eventsResult = (
                service.events()
                .list(
                    calendarId=currentCalendar,
                    timeMin=min_timestamp,
                    timeMax=max_timestamp,
                    maxResults=100,
                    singleEvents=True,
                    orderBy="startTime",
                )
                .execute()
            )

            events = eventsResult.get("items", [])

            for eventData in events:
                event = Event(
                    title=eventData.get("summary", ""),
                    description=eventData.get("description", ""),
                    location=eventData.get("location", ""),
                    full_day="date" in eventData.get("start", {}),
                    date=eventData.get("start", {}).get("date", ""),
                    startTime=eventData.get("start", {}).get("dateTime", ""),
                    endTime=eventData.get("end", {}).get("dateTime", ""),
                )
                eventArray.append(event)

    return eventArray


# Example usage:
# print(getEventsByTimeframe("2022-10-19T00:00:00.000001Z", "2022-10-19T23:59:59.999999Z"))

[Event(title='Test2', description='Beschreibung', location='Ortsname', full_day=True, date='2022-10-19', startTime='', endTime=''), Event(title='Test', description='', location='', full_day=False, date='', startTime='2022-10-19T20:00:00+02:00', endTime='2022-10-19T21:00:00+02:00'), Event(title='Maschinelles Lernen', description='', location='', full_day=False, date='', startTime='2022-10-19T08:30:00+02:00', endTime='2022-10-19T12:00:00+02:00'), Event(title='Maschinelles Lernen', description='', location='', full_day=False, date='', startTime='2022-10-19T13:00:00+02:00', endTime='2022-10-19T16:00:00+02:00')]


In [106]:
def getAllEventsToday():
    todayMin = datetime.today().strftime("%Y-%m-%dT00:00:00.000001Z")
    todayMax = datetime.today().strftime("%Y-%m-%dT23:59:59.999999Z")

    return getEventsByTimeframe(todayMin, todayMax)


# Example usage:
# print(getAllEventsToday())

[Event(title='Testeintrag', description='', location='', full_day=True, date='2022-11-03', startTime='', endTime=''), Event(title='Advanced Software Engineering', description='', location='', full_day=False, date='', startTime='2022-11-03T08:15:00+01:00', endTime='2022-11-03T12:00:00+01:00'), Event(title='Maschinelles Lernen - ONLINE', description='', location='', full_day=False, date='', startTime='2022-11-03T16:00:00+01:00', endTime='2022-11-03T17:30:00+01:00')]


In [107]:
def getNextEventToday():
    events = getAllEventsToday()

    for event in events:
        if not event.full_day: 
            startTime = datetime.strptime(event.startTime, "%Y-%m-%dT%H:%M:%S+01:00")
            if datetime.now() < startTime:
                return event 
    return None

# Example usage:
# print(getNextEventToday())

Event(title='Maschinelles Lernen - ONLINE', description='', location='', full_day=False, date='', startTime='2022-11-03T16:00:00+01:00', endTime='2022-11-03T17:30:00+01:00')


In [108]:
def createEvent(eventInfo: Event):
    if eventInfo.full_day:
        endDate = (datetime.fromisoformat(eventInfo.date) + timedelta(days=1)).strftime("%Y-%m-%d")
    else:
        endDate = ""

    event = {
        "summary": eventInfo.title,
        "location": eventInfo.location,
        "description": eventInfo.description,
        "start": {"dateTime": eventInfo.startTime, "timeZone": "Europe/Berlin", "date": eventInfo.date},
        "end": {"dateTime": eventInfo.endTime, "timeZone": "Europe/Berlin", "date": endDate},
    }

    if eventInfo.full_day:
        event.get("start").pop("dateTime")
        event.get("start").pop("timeZone")
        event.get("end").pop("dateTime")
        event.get("end").pop("timeZone")
    else:
        event.get("start").pop("date")
        event.get("end").pop("date")

    service = get_calendar_service()
    createdEvent = service.events().insert(calendarId="primary", body=event).execute()

    # print(f'Created Event: {event}')


# Example usage:
# createEvent(Event(
#     title = 'TestCreate1',
#     description = 'Description1',
#     location = 'Location1',
#     full_day = False,
#     date = '2022-10-20',
#     startTime = '2022-10-20T16:30:00+01:00',
#     endTime = '2022-10-20T17:30:00+01:00'
# ))

Created Event: {'summary': 'TestCreate1', 'location': 'Location1', 'description': 'Description1', 'start': {'dateTime': '2022-10-20T16:30:00+01:00', 'timeZone': 'Europe/Berlin'}, 'end': {'dateTime': '2022-10-20T17:30:00+01:00', 'timeZone': 'Europe/Berlin'}}
